# RAG

## Document Loader

In [ ]:
# Install the langchain_community package
# %pip install langchain_community
# %pip install unstructured

In [ ]:
# 1. PDF document loader

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("paht/to/file/attention_is_all_you_need.pdf") # Load the PDF file
data = loader.load() # Load the PDF file into memory
print(data[0])
print(loader.text) # Print the text of the PDF file


In [ ]:
# 2. CSV document loader

from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader("fifa_countries_audience.csv") # Load the PDF file

data = loader.load() # Load the PDF file into memory
print(data[0])

In [ ]:
# 3. HTML document loader

from langchain_community.document_loaders import UnstructuredHTMLLoader
# loader = UnstructuredHTMLLoader("https://en.wikipedia.org/wiki/Deep_learning") # Load the PDF file
loader = UnstructuredHTMLLoader("white_house_executive_order_nov_2023") # Load the PDF file

data = loader.load() # Load the PDF file into memory
print(data[0]) # Print the first document

# Print the first document's metadata
print(data[0].metadata)

## Document Splitting